# Table of Contents
- [ Contingency plan levels ](#_contingency plan levels ) 
  - [ Check SRAG dataset situation](#_check srag dataset situation) 
  - [ Check SRAGFlu dataset situation](#_check sragflu dataset situation) 
  - [ Check ObitoFlu dataset situation](#_check obitoflu dataset situation) 
- [ Concise test](#_concise test) 


# Contingency plan levels <a name="_contingency plan levels"></a>

Basal: white

Level 0: yellow <br>
    SRAG cases ascending for the last 4 weeks AND with at least one of them inside alert zone.

Level 1: orange <br>
    SRAGFlu cases ascending for the last 4 weeks AND with at least one of them inside alert zone.

Level 2: red <br>
    ObitosFlu cases ascending for the last 3 weeks AND with at least one of them inside alert zone.<a name="_contingency plan levels: "></a><a name="_contingency plan levels "></a><a name="_contingency plan levels "></a>

In [1]:
import pandas as pd
from fludashboard.libs.flu_data import FluDB
fluDB = FluDB()

In [2]:
territory_name = 'Rio Grande do Sul'
year=2016
territory_id = fluDB.get_territory_id_from_name(territory_name)
dataset_from_id = {
    1: 'srag',
    2: 'sragflu',
    3: 'obitoflu'
}

In [3]:
def contingency_alert(dataset_id: int, year: int, territory_id: int):
    df = fluDB.get_data(dataset_id=dataset_id, scale_id=1, year=year, territory_id=territory_id)
    
    # If not obitoflu dataset (3), uses last 4 weeks, o.w. use 3:
    if dataset_id < 3:
        wdw = 4
    else:
        wdw = 3
        
    alert_zone = any(df.estimated_cases[-wdw:] > df.typical_high[-wdw:])
    data_increase = all(df.estimated_cases[-wdw:].values - df.estimated_cases[-(wdw+1):-1].values > 0)
    
    print('''
    Data: %s
    Entered alert zone? %s
    Steady increase in the window of interest? %s
    Trigger alert? %s
    ''' % (dataset_from_id[dataset_id], alert_zone, data_increase, alert_zone & data_increase)
    )
    
    return(alert_zone & data_increase)

## Check SRAG dataset situation<a name="_check srag dataset situation"></a>

In [4]:
yellow_alert = contingency_alert(dataset_id=1, year=year, territory_id=territory_id)
yellow_alert


    Data: srag
    Entered alert zone? True
    Steady increase in the window of interest? False
    Trigger alert? False
    


False

## Check SRAGFlu dataset situation<a name="_check sragflu dataset situation"></a>

In [5]:
orange_alert = contingency_alert(dataset_id=2, year=year, territory_id=territory_id)
orange_alert


    Data: sragflu
    Entered alert zone? True
    Steady increase in the window of interest? False
    Trigger alert? False
    


False

## Check ObitoFlu dataset situation<a name="_check obitoflu dataset situation"></a>

In [6]:
red_alert = contingency_alert(dataset_id=3, year=year, territory_id=territory_id)
red_alert


    Data: obitoflu
    Entered alert zone? True
    Steady increase in the window of interest? False
    Trigger alert? False
    


False

# Concise test<a name="_concise test"></a>

In [7]:
contingency_name_from_id = {
    0: 'Nível basal',
    1: 'Nível 0',
    2: 'Nível 1',
    3: 'Nível 2',    
}

def alert_trigger(dataset_id: int, year: int, territory_id: int):
    df = fluDB.get_data(dataset_id=dataset_id, scale_id=1, year=year, territory_id=territory_id)

    # If not obitoflu dataset (3), uses last 4 weeks, o.w. use 3:
    if dataset_id < 3:
        wdw = 4
    else:
        wdw = 3

    alert_zone = any(df.estimated_cases[-wdw:] > df.typical_high[-wdw:])
    data_increase = all(df.estimated_cases[-wdw:].values - df.estimated_cases[-(wdw+1):-1].values > 0)

    return(alert_zone & data_increase)


def contingency_level(year: int, territory_id: int):
    
    if alert_trigger(dataset_id=3, year=year, territory_id=territory_id):
        return(3)
    elif alert_trigger(dataset_id=2, year=year, territory_id=territory_id):
        return(2)
    elif alert_trigger(dataset_id=1, year=year, territory_id=territory_id):
        return(1)
    else:
        return(0)

In [8]:
contingency_name_from_id[contingency_level(year=year, territory_id=territory_id)]

'Nível basal'